In [ ]:
## Notebook for Creating Generator code for Keras and PyTorch
from __future__ import print_function, division
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
#
# Import base tools

## Note, for mac osx compatability import something from shapely.geometry before importing fiona or geopandas
## https://github.com/Toblerity/Shapely/issues/553  * Import shapely before rasterio or fioana
from shapely import geometry
import rasterio
import random
from cw_tiler import main
from cw_tiler import utils
from cw_tiler import vector_utils
import numpy as np
import os
from tqdm import tqdm
import random
# Setting Certificate Location for Ubuntu/Mac OS locations (Rasterio looks for certs in centos locations)
## TODO implement os check before setting
os.environ['CURL_CA_BUNDLE']='/etc/ssl/certs/ca-certificates.crt'

In [ ]:
class largeGeoTiff(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, rasterPath, 
                 stride_size_meters=150,
                 cell_size_meters = 200,
                 tile_size_pixels = 650, 
                 transform=None):
        """
        Args:
            rasterPath (string): Path to the rasterFile
            stride_size_meters (float): sliding window stride size in meters
            cell_size_meters (float): sliding window size in meters
            tile_size_pixels (float): sliding window pixel dimensions
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.rasterPath = rasterPath
        self.stride_size_meters = stride_size_meters
        self.cell_size_meters = cell_size_meters
        self.tile_size_pixels = tile_size_pixels
        self.transform = transform
        
        self.src = rasterio.open(spacenetPath)

        # Get Lat, Lon bounds of the Raster (src)
        self.wgs_bounds = utils.get_wgs84_bounds(self.src)

        # Use Lat, Lon location of Image to get UTM Zone/ UTM projection
        self.utm_crs = utils.calculate_UTM_crs(self.wgs_bounds)

        # Calculate Raster bounds in UTM coordinates 
        self.utm_bounds = utils.get_utm_bounds(self.src, utm_crs)
        
        self.rasterBounds = geometry.box(*self.utm_bounds)
        self.cells_list = main.calculate_analysis_grid(self.rasterBounds.bounds, 
                                                       stride_size_meters=self.stride_size_meters, 
                                                       cell_size_meters=self.cell_size_meters
                                                      )
        



            

    def __len__(self):
        return len(self.cells_list)

    def __getitem__(self, idx):
        cell_selection = cells_list[idx]
        ll_x, ll_y, ur_x, ur_y = cell_selection


        # Get Tile from bounding box
        tile, mask, window, window_transform = main.tile_utm(self.src, ll_x, ll_y, ur_x, ur_y, indexes=None, tilesize=tile_size_pixels, nodata=None, alpha=None,
                         dst_crs=self.utm_crs)
        
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'tile': tile, 
                  'mask': mask,
                 'window': window,
                 'window_transform': window_transform}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
def pad(img, pad_size=32):
    """
    Load image from a given path and pad it on the sides, so that eash side is divisible by 32 (network requirement)
    if pad = True:
        returns image as numpy.array, tuple with padding in pixels as(x_min_pad, y_min_pad, x_max_pad, y_max_pad)
    else:
        returns image as numpy.array
    """

    if pad_size == 0:
        return img

    height, width = img.shape[:2]

    if height % pad_size == 0:
        y_min_pad = 0
        y_max_pad = 0
    else:
        y_pad = pad_size - height % pad_size
        y_min_pad = int(y_pad / 2)
        y_max_pad = y_pad - y_min_pad

    if width % pad_size == 0:
        x_min_pad = 0
        x_max_pad = 0
    else:
        x_pad = pad_size - width % pad_size
        x_min_pad = int(x_pad / 2)
        x_max_pad = x_pad - x_min_pad

    img = cv2.copyMakeBorder(img, y_min_pad, y_max_pad, x_min_pad, x_max_pad, cv2.BORDER_REFLECT_101)

    return img, (x_min_pad, y_min_pad, x_max_pad, y_max_pad)

def minmax(img):
    out = np.zeros_like(img).astype(np.float32)
    if img.sum() == 0:
        return out

    for i in range(img.shape[2]):
        c = img[:, :, i].min()
        d = img[:, :, i].max()

        t = (img[:, :, i] - c) / (d - c)
        out[:, :, i] = t
    return out.astype(np.float32)

def reform_tile(tile, rollaxis=True):
    
    if rollaxis:
        tile = np.rollaxis(tile, 0,3)
    rgb = minmax(tile[:,:,(5,3,2)]) 
    
    tf = tile.astype(np.float32)/ (2**11 - 1)
    
    return np.concatenate([rgb, tf], axis=2) * (2**8 - 1)

def teranaus_transform(sample):
    """sample = {'tile': tile, 
                  'mask': mask,
                 'window': window,
                 'window_transform': window_transform}
                 
                 """
    
    img = reform_tile(tile)
    img, pads = pad(img)
    input_img = torch.unsqueeze(img_transform(img / 255).cuda(), dim=0)

    